Output the precipitation anomalies in map form and averaged over the three regions used in the paper (US west coast, US south and Southern Mexico),
Past= 1979-2005 of historical concatenated with 2006-2014 of RCP8.5 and Future = 2070-2099 of RCP8.5.

Output is put to ecpaper2020/DATASORT/VWIND/DATA/

In [15]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import warnings

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import shapefile_utils as shp

from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)
importlib.reload(shp)

warnings.filterwarnings('ignore')

Set paths for CMIP5 models and output

In [2]:
histpath="/project/cmip5/historical/Amon/"
rcp85path="/project/cmip5/rcp85/Amon/"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Set up shapefiles

In [8]:
#USA
shpdir = "/project/cas/islas/python/ecpaper2020/shapefiles/USA/"
shpurl = "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_USA_shp.zip"
with urlopen(shpurl) as ziploc:
    with ZipFile(BytesIO(ziploc.read())) as zfile:
        zfile.extractall(shpdir)
shpfile_USA = shpdir+"gadm36_USA_1.shp"

# Mexico
shpdir = "/project/cas/islas/python/ecpaper2020/shapefiles/Mexico/"
shpurl = "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_MEX_shp.zip"
with urlopen(shpurl) as ziploc:
    with ZipFile(BytesIO(ziploc.read())) as zfile:
        zfile.extractall(shpdir)
shpfile_MEX = shpdir+"gadm36_MEX_0.shp"


Information on the models is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [3]:
cmip5models=pd.read_csv("../cmip5csvinfo.csv")
ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for past period 
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp = str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp = str(yendp)+"-"+str(monendp).zfill(2)
datebegf = str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf = str(yendf)+"-"+str(monendf).zfill(2)

In [83]:
models = cmip5models["Model"]
nmodels=models.size

# define 1 deg grid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi-model arrays
prpast_em = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='prpast_em')
prfut_em = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='prfut_em')

pr_westmask_1memp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_westmask_1memp')
pr_southmask_1memp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_southmask_1memp')
pr_mexmask_1memp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_mexmask_1memp')

pr_westmask_1memf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_westmask_1memf')
pr_southmask_1memf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_southmask_1memf')
pr_mexmask_1memf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_mexmask_1memf')

pr_westmask_emp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_westmask_emp')
pr_southmask_emp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_southmask_emp')
pr_mexmask_emp = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_mexmask_emp')

pr_westmask_emf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_westmask_emf')
pr_southmask_emf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_southmask_emf')
pr_mexmask_emf = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='pr_mexmask_emf')


# loop over models
for index, modname in models.iteritems():
    
    # initialize boolean for reusing weights
    wgtfile = "/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/tmp/wgt_pr_cmip5.nc"
    reusewgt = False
    
    # --- the past period ---
    nmems = cmip5models.loc[index, "Nmempast"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        memstr="r"+str(imem)+"i1p1"
        
        # check for a special order
        changeorder = cmip5models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            memstr = "r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir = histpath+"pr/"+modname+"/"+memstr+"/"
        rcp85dir = rcp85path+"pr/"+modname+"/"+memstr+"/"
        
        prhist = read.read_sfc(histdir+"/*.nc", datebegp, "2005-12")
        prrcp = read.read_sfc(rcp85dir+"/*.nc", "2006-01", dateendp)
        pr = xr.concat([prhist, prrcp], dim="time", join="override")
        
        # check the array length.  If this fails, try ending historical in 2005-11
        # and beginning rcp in 2005-12 to fix Hadley center
        if (pr.time.size != nmonthsp):
            print("switching to the assumption that historical ends in 2005-11")
            prhist = read.read_sfc(histdir+"/*.nc", datebegp, "2005-11")
            prrcp = read.read_sfc(rcp85dir+"/*.nc", "2005-12", dateendp)
            pr = xr.concat([prhist, prrcp], dim="time", join="override")
            
        # check again
        if (pr.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but pr has size "+str(pr.time.size))
            sys.exit("pr past for "+modname+", "+memstr+" doesn't have the right size")
            
        prdjf = cal.season_mean(pr, "pr", season="DJF")
        prdjf = 86400.*prdjf
        regridder = xe.Regridder(prdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        prdjfinterp = regridder(prdjf)
        
        reusewgt=True
        
        prpast_em[index,:,:] = prpast_em[index,:,:] + prdjfinterp[:,:]/float(nmems)
        
        if (index == 0) and (imem == 1): # only need to make the mask for 1deg once
            maskwest = shp.maskgen(shpfile_USA, prdjfinterp, ['California','Oregon','Washington'])
            masksouth = shp.maskgen(shpfile_USA, prdjfinterp, 
                                ['Texas', 'New Mexico', 'Louisiana', 'Alabama', 'Mississippi', 'Oklahoma', 'Arkansas'])
            maskmex = shp.maskgen(shpfile_MEX, prdjfinterp,['Mexico'])
            lat2d = np.repeat(np.expand_dims(maskmex.lat, axis=1), maskmex.lon.size, axis=1)
            maskmex = np.where(lat2d > 22, nan, maskmex)
            maskmex = xr.DataArray(maskmex, coords = grid_out.coords)
            
        pr_westmask = xr.DataArray(np.array(prdjfinterp) * np.array(maskwest), coords=prdjfinterp.coords)
        pr_southmask = xr.DataArray(np.array(prdjfinterp) * np.array(masksouth), coords=prdjfinterp.coords)
        pr_mexmask = xr.DataArray(np.array(prdjfinterp)*np.array(maskmex), coords=prdjfinterp.coords)

        if (imem == 1):
            pr_westmask_1memp[index,:,:] = pr_westmask[:,:]
            pr_southmask_1memp[index,:,:] = pr_southmask[:,:]
            pr_mexmask_1memp[index,:,:] = pr_mexmask[:,:]
            
        pr_westmask_emp[index,:,:]=pr_westmask_emp[index,:,:] + pr_westmask[:,:]/float(nmems)
        pr_southmask_emp[index,:,:]=pr_southmask_emp[index,:,:] + pr_southmask[:,:]/float(nmems)
        pr_mexmask_emp[index,:,:]=pr_mexmask_emp[index,:,:] + pr_mexmask[:,:]/float(nmems)
        
        
    # --- the future period
    
    nmems = cmip5models.loc[index, "Nmemfuture"]
    for imem in range(1, nmems+1, 1): # loop over future members
        memstr = 'r'+str(imem)+'i1p1'
        changeorder = cmip5models.loc[index, 'specialorderfuture']
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing future for "+modname+" "+memstr)
        
        rcp85dir = rcp85path+"pr/"+modname+"/"+memstr+"/"
        
        pr = read.read_sfc(rcp85dir+"/*.nc", datebegf, dateendf) 
        
        if (pr.time.size != nmonthsf):
            print("Somthing's wrong, nmonthsf="+str(nmonthsf)+" but pr has size "+str(pr.time.size))
            sys.exit("failed at "+modname+" "+memstr)
            
        prdjf = cal.season_mean(pr, "pr", season="DJF")
        prdjf = 86400.*prdjf
        regridder = xe.Regridder(prdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        prdjfinterp = regridder(prdjf)
        
        reusewgt=True
        
        prfut_em[index,:,:] = prfut_em[index,:,:] + prdjfinterp[:,:]/float(nmems)
            
        pr_westmask = xr.DataArray(np.array(prdjfinterp) * np.array(maskwest), coords=prdjfinterp.coords)
        pr_southmask = xr.DataArray(np.array(prdjfinterp) * np.array(masksouth), coords=prdjfinterp.coords)
        pr_mexmask = xr.DataArray(np.array(prdjfinterp)*np.array(maskmex), coords=prdjfinterp.coords)

        if (imem == 1):
            pr_westmask_1memf[index,:,:] = pr_westmask[:,:]
            pr_southmask_1memf[index,:,:] = pr_southmask[:,:]
            pr_mexmask_1memf[index,:,:] = pr_mexmask[:,:]
                       
        pr_westmask_emf[index,:,:]=pr_westmask_emf[index,:,:] + pr_westmask[:,:]/float(nmems)
        pr_southmask_emf[index,:,:]=pr_southmask_emf[index,:,:] + pr_southmask[:,:]/float(nmems)
        pr_mexmask_emf[index,:,:]=pr_mexmask_emf[index,:,:] + pr_mexmask[:,:]/float(nmems)           
        

            

        
        
        
        
        

Processing past for ACCESS1-0 r1i1p1...
masking California
masking Oregon
masking Washington
masking Texas
masking New Mexico
masking Louisiana
masking Alabama
masking Mississippi
masking Oklahoma
masking Arkansas
masking Mexico
Processing future for ACCESS1-0 r1i1p1
Processing past for ACCESS1-3 r1i1p1...
Processing future for ACCESS1-3 r1i1p1
Processing past for bcc-csm1-1 r1i1p1...
Processing future for bcc-csm1-1 r1i1p1
Processing past for bcc-csm1-1-m r1i1p1...
Processing future for bcc-csm1-1-m r1i1p1
Processing past for BNU-ESM r1i1p1...
Processing future for BNU-ESM r1i1p1
Processing past for CanESM2 r1i1p1...
Processing past for CanESM2 r2i1p1...
Processing past for CanESM2 r3i1p1...
Processing past for CanESM2 r4i1p1...
Processing past for CanESM2 r5i1p1...
Processing future for CanESM2 r1i1p1
Processing future for CanESM2 r2i1p1
Processing future for CanESM2 r3i1p1
Processing future for CanESM2 r4i1p1
Processing future for CanESM2 r5i1p1
Processing past for CCSM4 r1i1p1...
P

In [90]:
prwest_p_em = avg.cosweightlonlat(pr_westmask_emp, 0, 360, -90, 90)
prwest_p_em = prwest_p_em.rename("prwest_p_em")
prwest_f_em = avg.cosweightlonlat(pr_westmask_emf, 0, 360, -90, 90)
prwest_f_em = prwest_f_em.rename("prwest_f_em")
prwest_p_1mem = avg.cosweightlonlat(pr_westmask_1memp, 0, 360, -90, 90)
prwest_p_1mem = prwest_p_1mem.rename("prwest_p_1mem")
prwest_f_1mem = avg.cosweightlonlat(pr_westmask_1memf, 0, 360, -90, 90)
prwest_f_1mem = prwest_f_1mem.rename("prwest_f_1mem")

prsouth_p_em = avg.cosweightlonlat(pr_southmask_emp, 0, 360, -90, 90)
prsouth_p_em = prsouth_p_em.rename("prsouth_p_em")
prsouth_f_em = avg.cosweightlonlat(pr_southmask_emf, 0, 360, -90, 90)
prsouth_f_em = prsouth_f_em.rename("prsouth_f_em")
prsouth_p_1mem = avg.cosweightlonlat(pr_southmask_1memp, 0, 360, -90, 90)
prsouth_p_1mem = prsouth_p_1mem.rename("prsouth_p_1mem")
prsouth_f_1mem = avg.cosweightlonlat(pr_southmask_1memf, 0, 360, -90, 90)
prsouth_f_1mem = prsouth_f_1mem.rename("prsouth_f_1mem")

prmex_p_em = avg.cosweightlonlat(pr_mexmask_emp, 0, 360, -90, 90)
prmex_p_em = prmex_p_em.rename("prmex_p_em")
prmex_f_em = avg.cosweightlonlat(pr_mexmask_emf, 0, 360, -90, 90)
prmex_f_em = prmex_f_em.rename("prmex_f_em")
prmex_p_1mem = avg.cosweightlonlat(pr_mexmask_1memp, 0, 360, -90, 90)
prmex_p_1mem = prmex_p_1mem.rename("prmex_p_1mem")
prmex_f_1mem = avg.cosweightlonlat(pr_mexmask_1memf, 0, 360, -90, 90)
prmex_f_1mem = prmex_f_1mem.rename("prmex_f_1mem")

prdata = xr.merge([prwest_p_em, prwest_f_em, prwest_p_1mem, prwest_f_1mem,
                 prsouth_p_em, prsouth_f_em, prsouth_p_1mem, prsouth_f_1mem,
                 prmex_p_em, prmex_f_em, prmex_p_1mem, prmex_f_1mem])

prdata.to_netcdf(path=pathout+"cmip5vwind_prdata.nc")

prdif = prfut_em - prpast_em
prdif.to_netcdf(path=pathout+"/cmip5vwind_prmapdata.nc")
